Opsmate has a handful of [built-in](/cookbooks/automation-using-python-runtime/#built-in-tools) tools you can use out of the box, however there are always customisations and extensions you want to build for your own use cases.

Opsmate comes with a plugin system to allow you to build your own tools and use them in your automation.

In this cookbook, we will show you how to write and author your own plugins and use it in your automation.



## Prerequisites

* You have a OpenAI API key, otherwise Anthropic API key is also supported, as Opsmate is LLM provider agnostic.
* You have Opsmate installed - see [getting started](/#getting-started) for more details.
* You have [kind](https://kind.sigs.k8s.io/docs/user/quick-start/#installation) installed, as Opsmate uses kind to run local Kubernetes cluster.
* You have [kubectl](https://kubernetes.io/docs/tasks/tools/#kubectl) installed for interacting with the local Kubernetes cluster.

## Setup

First, let's install our required packages and set our API keys.

In [1]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("OPENAI_API_KEY") # Feel to comment this out and use Anthropic API key instead
_set_if_undefined("ANTHROPIC_API_KEY")


Then we will spin up a local k8s cluster and install lightweight LGTM stack on it for testing.

In [4]:
! kind create cluster --name opsmate-plugin-test

Creating cluster "opsmate-plugin-test" ...
 ✓ Ensuring node image (kindest/node:v1.31.2) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-opsmate-plugin-test"
You can now use your cluster with:

kubectl cluster-info --context kind-opsmate-plugin-test

Thanks for using kind! 😊


In [6]:
# make sure that you are running on the correct cluster
! kubectl config current-context

kind-opsmate-plugin-test


Now we will install the LGTM stack on the cluster.

In [7]:
! kubectl apply -f https://raw.githubusercontent.com/grafana/docker-otel-lgtm/refs/heads/main/k8s/lgtm.yaml

service/lgtm created
deployment.apps/lgtm created


In [8]:
# make sure that the lgtm stack is running
! kubectl wait --for=condition=available deployment/lgtm

deployment.apps/lgtm condition met


## Our first plugin

The `./plugins/loki.py` file contains the plugin we wrote for this cookbook. The job it tries to accomplish is reasonably complex:

* It extracts the datetime range from the user's query
* It queries the loki API to get the logs within the datetime range
* It returns the logs to a string and represent it as a markdown table



In [10]:
! cat ./plugins/loki.py

from opsmate.dino.types import ToolCall, PresentationMixin
from pydantic import Field, PrivateAttr
from typing import Literal, ClassVar, Optional
from httpx import AsyncClient
import os
import base64
from opsmate.dino import dino
from opsmate.dino.types import Message
from opsmate.tools.datetime import DatetimeRange, datetime_extraction
from opsmate.plugins import auto_discover


class LokiBase(ToolCall, PresentationMixin):
    """
    A tool to query logs in loki
    """

    user_id: ClassVar[str] = None
    auth_token: ClassVar[str] = None

    limit: int = Field(description="The number of results to return", default=100)
    direction: Literal["forward", "backward"] = Field(
        description="The direction of the search", default="forward"
    )
    output: Optional[str] = Field(
        description="The output of the loki query - DO NOT USE THIS FIELD",
        default=None,
    )

    _client: AsyncClient = PrivateAttr(default_factory=AsyncClient)

    def headers(self):
     

## Discovering plugins

In the plugin code if you have a keen eye you probably have already noticed the `@auto_discover` decorator. This is the key to discover the plugin.

By default all the tools are discovered automatically, but for LLM functions you will need to explicitly mark it as discoverable via the `@auto_discover` decorator.

To make the plugin discovered from where-ever your current python path is, you can execute the discovery via the following snippet:

In [17]:
from opsmate.plugins import PluginRegistry as plugins

plugins.clear()
plugins.discover("./plugins", ignore_conflicts=True)


2025-01-14 17:11:36 [debug    ] loading builtin tools         
2025-01-14 17:11:36 [debug    ] loading builtin tools from     builtin_module=opsmate.tools
2025-01-14 17:11:36 [debug    ] loading dtool                  dtool=FileAppend
2025-01-14 17:11:36 [warning  ] tool already exists            conflict_source=/home/jingkaihe/workspace/opsmate/opsmate/tools/__init__.py tool=FileAppend
2025-01-14 17:11:36 [debug    ] loading dtool                  dtool=FileDelete
2025-01-14 17:11:36 [warning  ] tool already exists            conflict_source=/home/jingkaihe/workspace/opsmate/opsmate/tools/__init__.py tool=FileDelete
2025-01-14 17:11:36 [debug    ] loading dtool                  dtool=FileRead
2025-01-14 17:11:36 [warning  ] tool already exists            conflict_source=/home/jingkaihe/workspace/opsmate/opsmate/tools/__init__.py tool=FileRead
2025-01-14 17:11:36 [debug    ] loading dtool                  dtool=FileWrite
2025-01-14 17:11:36 [warning  ] tool already exists            co

/home/jingkaihe/workspace/opsmate/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
